In [1]:
# Imports
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
def qr_rank(A, tol=1e-9):
    Q, R = np.linalg.qr(A)
    rank = np.sum(np.abs(np.diag(R)) > tol)
    row_indices = np.argsort(np.abs(np.diag(R)))[::-1][:rank]
    return row_indices

In [7]:
A = np.array([[1, 2, 3,1],
              [4, 5, 6,1],
              [2, 8, 9,2]])

In [8]:
qr_rank(A)

array([1, 0, 2], dtype=int64)

In [20]:
np.zeros((3, 4)).dtype

dtype('float64')

In [26]:
A = np.array([0.1, 0.2, 0.3, 0.4], dtype=np.float64)
B = np.array([60000, 500, 600, 2], dtype=np.uint16)
C = np.array([1, 2, 3, 4], dtype=np.uint8)
np.dot(A, B)
B+C

array([60001,   502,   603,     6], dtype=uint16)

In [10]:
A @ B.T

33

In [3]:
a = np.array([[1, 2, 3, 4, 5], [1,1,1,1,1]])

In [5]:
np.sum(a, axis=0)

array([2, 3, 4, 5, 6])

In [10]:
np.nonzero(a)

(array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int64),
 array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4], dtype=int64))

In [13]:
b = np.array([1, 2, 3, 4, 5, 0, 0, 0, 0, 0])   

In [16]:
len(np.nonzero(b)[0])

5

In [81]:
'''
Simulation Class simulates the experiment environment for Reinforcement Learning agent
Arguments:
    - n_maxstep: the maximum number of steps the agent can take in the simulation
    - dictionary: each row represents a different electrode-amplitude pair. Each column represents a activation probability of different cell.
    - n_elecs: the number of electrodes
    - n_amps: the number of amplitudes
    - elecs: the electrode numbers corresponding to the rows of the dictionary
    - amps: the amplitudes corresponding to the rows of the dictionary
    - elec_map: maps electrode numbers to their locations on the brain
    - cell_ids: the cell ids corresponding to the columns of the dictionary

Variables:
    - n_step: the current number of steps taken in the simulation
    - elec: the current electrode
    - amp: the current amplitude
    - done: whether the episode is done
    - reward: the reward for the current step
    - state: the current state of the simulation
    
Functions:
    - __init__: initializes the simulation
    - reset: resets the simulation to the initial state
    - step: takes in an action and returns the next state, reward, and whether the episode is done
    - sample: samples cell activations using the probability specified in the dictionary
    - render: renders the current state of the simulation
    - close: closes the simulation
'''
class SimulationEnv:
    def __init__(self, path, reward_func, score_func, n_maxstep, n_elecs, n_amps):
        # Load relevant data from .npz files
        try:
            with np.load(os.path.join(path,"dictionary.npz")) as data:
                self.dict = data["dictionary"]
                self.elecs = data["entry_elecs"]
                self.amps = data["entry_amps"]
                self.elec_map = data["elec_map"]
            with np.load(os.path.join(path,"decoders.npz")) as data:
                self.cell_ids = data["cell_ids"]
        except FileNotFoundError:
            print("Please make sure the dictionary.npz and decoders.npz files are present in the specified path")

        # Initialize variables
        self.reward_func = reward_func
        self.score_func = score_func
        self.n_maxstep = n_maxstep
        self.n_elecs = n_elecs
        self.n_amps = n_amps
        self.n_cells = len(self.cell_ids)
        self.reset()
        
    def reset(self):
        # Reset variables
        self.n_step = 0
        self.elec = 0 # electrode number (1~n_elecs)
        self.amp = 0 # amplitude (1~n_amps)
        self.done = False
        self.reward = 0
        self.dict_hat = np.zeros((self.n_elecs*self.n_amps, len(self.cell_ids)), dtype=np.uint16)
        self.state = 0
        return self.state
    
    def step(self, action):
        self.elec = action[0]
        self.amp = action[1]

        if self.n_step >= self.n_maxstep:
            self.done = True
        else:
            sampled_activations = self.sample(self.elec, self.amp)
            self.dict_hat[(self.elec-1)*self.n_amps + (self.amp-1)] += sampled_activations
            self.state = self.score_func(self.dict_hat)
            self.reward = self.reward_func(sampled_activations)
            self.n_step += 1
        return self.state, self.reward, self.done
    
    def sample(self, elec, amp):
        try:
            idx = np.where((self.elecs == elec) & (self.amps == amp))[0][0]
            dist = self.dict[idx]
        except IndexError:
            # print(f"Electrode {elec} with amplitude {amp} was not in the dictionary")
            # print(f"Assume no cells were activated")
            dist = np.zeros(len(self.cell_ids), dtype=np.float64)

        if np.any(dist < 0):
            invalid_idx = np.where(dist < 0)[0]
            print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 0
        if np.any(dist > 1):
            invalid_idx = np.where(dist > 1)[0]
            print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 1
        if np.any(np.isnan(dist)):
            invalid_idx = np.where(np.isnan(dist))[0]
            print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 0

        # if np.any(dist < 0) or np.any(dist > 1) or np.any(np.isnan(dist)):
        #     print(f"Invalid distribution: {dist}")
        #     dist = np.zeros(len(self.cell_ids), dtype=np.float64)
        sampled_activations = np.random.binomial(1, dist).astype(dtype=np.uint8)

        return sampled_activations
    
    def render(self, elec, amp):
        print(self.dict_hat[(elec-1)*self.n_amps + (amp-1)])
    
    def close(self):
        pass

In [82]:
class FullStateSimulationEnv:
    def __init__(self, path, reward_func, score_func, n_maxstep, n_elecs, n_amps):
        # Load relevant data from .npz files
        try:
            with np.load(os.path.join(path,"dictionary.npz")) as data:
                self.dict = data["dictionary"]
                self.elecs = data["entry_elecs"]
                self.amps = data["entry_amps"]
                self.elec_map = data["elec_map"]
            with np.load(os.path.join(path,"decoders.npz")) as data:
                self.cell_ids = data["cell_ids"]
        except FileNotFoundError:
            print("Please make sure the dictionary.npz and decoders.npz files are present in the specified path")

        # Initialize variables
        self.reward_func = reward_func
        self.score_func = score_func
        self.n_maxstep = n_maxstep
        self.n_elecs = n_elecs
        self.n_amps = n_amps
        self.n_cells = len(self.cell_ids)
        self.reset()
        
    def reset(self):
        # Reset variables
        self.n_step = 0
        self.elec = 0 # electrode number (1~n_elecs)
        self.amp = 0 # amplitude (1~n_amps)
        self.done = False
        self.reward = 0
        self.state = np.zeros((self.n_elecs*self.n_amps, len(self.cell_ids)), dtype=np.uint16)
        return self.state
    
    def step(self, action):
        self.elec = action[0]
        self.amp = action[1]

        if self.n_step >= self.n_maxstep:
            self.done = True
        else:
            sampled_activations = self.sample(self.elec, self.amp)
            self.state[(self.elec-1)*self.n_amps + (self.amp-1)] += sampled_activations
            # self.state = self.score_func(self.state)
            self.reward = self.reward_func(sampled_activations)
            self.n_step += 1
        return self.state, self.reward, self.done
    
    def sample(self, elec, amp):
        try:
            idx = np.where((self.elecs == elec) & (self.amps == amp))[0][0]
            dist = self.dict[idx]
        except IndexError:
            # print(f"Electrode {elec} with amplitude {amp} was not in the dictionary")
            # print(f"Assume no cells were activated")
            dist = np.zeros(len(self.cell_ids), dtype=np.float64)

        if np.any(dist < 0):
            invalid_idx = np.where(dist < 0)[0]
            # print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 0
        if np.any(dist > 1):
            invalid_idx = np.where(dist > 1)[0]
            # print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 1
        if np.any(np.isnan(dist)):
            invalid_idx = np.where(np.isnan(dist))[0]
            # print(f"Invalid value at index {invalid_idx}: {dist[invalid_idx]}")
            dist[invalid_idx] = 0

        # if np.any(dist < 0) or np.any(dist > 1) or np.any(np.isnan(dist)):
        #     print(f"Invalid distribution: {dist}")
        #     dist = np.zeros(len(self.cell_ids), dtype=np.float64)
        sampled_activations = np.random.binomial(1, dist).astype(dtype=np.uint8)

        return sampled_activations
    
    def render(self, elec, amp):
        print(self.state[(elec-1)*self.n_amps + (amp-1)])
    
    def close(self):
        pass

In [83]:
'''
Reward func calculates the reward for the agent
'''
def inverse_reward_func(array):
    if np.sum(array) == 1:
        return 1
    elif np.sum(array) == 0:
        return 0
    else:
        return 1/np.sum(array)

def span_reward_function(array):
    pass

In [84]:
'''
Score func calculates how well the produced dictionary is
Score is between 0 and the number of cells
'''
def span_score_func(vectors):
    # calculate the span of a set of vectors
    # :param vectors: list of vectors
    # :return: span of vectors
    span = np.linalg.matrix_rank(vectors)
    return span

def cosine_sim_score_func(vectors):
    # calculate the cosine similarity of a set of vectors
    # :param vectors: list of vectors
    # :return: cosine similarity of vectors
    sim = np.dot(vectors, vectors.T)
    return sim

In [85]:
# Eplison Greedy Agent
'''
Epsilon Greedy Agent takes in a simulation environment and uses epsilon greedy policy to find the optimal policy
'''

class EpsilonGreedyAgent:
    def __init__(self, env, epsilon=0.1, gamma=0.9, alpha=0.1):
        self.env = env
        self.epsilon = epsilon
        self.gamma = gamma
        self.alpha = alpha
        self.Q = np.zeros((env.n_cells, env.n_elecs*env.n_amps), dtype=np.float32)
        self.n = np.zeros((env.n_cells, env.n_elecs*env.n_amps), dtype=np.uint16)
        self.policy = np.zeros((env.n_cells), dtype=np.uint16)
        self.reset()

    def reset(self):
        self.state = self.env.reset()
        self.action = self.policy[self.state]
        return self.state, self.action
    
    def get_action(self, state):
        if np.random.random() < self.epsilon:
            action_idx = np.random.randint(0, self.env.n_elecs*self.env.n_amps)
        else:
            action_idx = np.argmax(self.Q[state])
            
        # convert action_idx to (elec, amp)
        action = (action_idx//self.env.n_amps + 1, action_idx%self.env.n_amps + 1)
        return action
    
    def step(self):
        self.action = self.get_action(self.state)
        self.s_next, self.reward,self.done = self.env.step(self.action)
        return self.s_next, self.reward, self.done
    
    def update(self):
        self.n[self.state, self.action] += 1
        self.Q[self.state, self.action] += self.alpha*(self.reward + self.gamma*np.max(self.Q[self.s_next]))
        self.policy[self.state] = np.argmax(self.Q[self.state])
        self.state = self.s_next
        return self.state, self.action
    
    def run(self, n_episodes=1000):
        self.reset()
        for i in range(n_episodes):
            # while not self.done:
            self.step()
            self.update()
        return self.policy
    
    def render(self):
        pass


In [86]:
# Stateless Eplison Greedy Agent
class StatelessEpsilonGreedyAgent:
    def __init__(self, env, epsilon=0.8, gamma=0.9, alpha=0.1):
        self.env = env
        self.epsilon = epsilon
        self.gamma = gamma
        self.alpha = alpha
        self.Q = np.zeros((env.n_elecs*env.n_amps), dtype=np.float32)
        self.n = np.zeros((env.n_elecs*env.n_amps), dtype=np.uint16)
        self.reset()

    def reset(self):
        self.action = self.get_action()
        return self.action
    
    def get_action(self):
        if np.random.random() < self.epsilon:
            action_idx = np.random.randint(0, self.env.n_elecs*self.env.n_amps)
        else:
            action_idx = np.argmax(self.Q)
        return action_idx

    def map_action(self, action_idx):
        # convert action_idx to (elec, amp)
        action = (action_idx//self.env.n_amps + 1, action_idx%self.env.n_amps + 1)
        return action
    
    def step(self):
        self.action = self.get_action()
        _, self.reward,self.done = self.env.step(self.map_action(self.action))
    
    def update(self):
        self.n[self.action] += 1
        self.Q[self.action] = self.reward + self.gamma*np.max(self.Q[self.action])
    
    def run(self, n_episodes=1000):
        self.reset()
        for i in range(n_episodes):
            # while not self.done:
            self.step()
            self.update()
        return self.Q
    
    def render(self):
        pass


In [117]:
# Thompson Sampling Agent
class StatelessTSAgend:
    def __init__(self, env, gamma=0.9):
        self.env = env
        self.gamma = gamma
        self.Q = np.zeros((env.n_elecs*env.n_amps), dtype=np.float32)
        self.n = np.zeros((env.n_elecs*env.n_amps), dtype=np.float32)
        self.reset()

    def reset(self):
        self.action = self.get_action()
        return self.action
    
    def get_action(self):
        prob_dist = np.copy(self.Q) + 1.0
        Q_sum =np.sum(prob_dist)
        if Q_sum == 0:
            action_idx = np.random.randint(0, self.env.n_elecs*self.env.n_amps)
        else:   
            prob_dist=prob_dist/Q_sum
            action_idx = np.random.choice(len(self.Q), size=1, p=prob_dist)
        return action_idx

    def map_action(self, action_idx):
        # convert action_idx to (elec, amp)
        action = (action_idx//self.env.n_amps + 1, action_idx%self.env.n_amps + 1)
        return action
    
    def step(self):
        self.action = self.get_action()
        _, self.reward,self.done = self.env.step(self.map_action(self.action))
    
    def update(self):
        self.n[self.action] += 1
        self.Q[self.action] = self.reward + self.gamma*np.max(self.Q[self.action])
    
    def run(self, n_episodes=1000):
        self.reset()
        for i in range(n_episodes):
            # while not self.done:
            self.step()
            self.update()
        return self.Q
    
    def render(self):
        pass

In [7]:
experiments = ["2022-11-04-2", "2022-11-28-1"]
path = f"./data/{experiments[0]}/dictionary"

In [119]:
sim = FullStateSimulationEnv(path, reward_func=inverse_reward_func, score_func=span_score_func, n_maxstep=100000, n_elecs=512, n_amps=42)

In [120]:
agent = StatelessTSAgend(sim)
Q_value = agent.run(n_episodes=50000)

In [121]:
def display_non_zero(array):
    # array is a 2D numpy array
    non_zero_entries = np.nonzero(array)
    print("Number of non-zero entries: {}".format(len(non_zero_entries[0])))

    # sort non-zero entries by value
    non_zero_entries = zip(*sorted(zip(*non_zero_entries), key=lambda x: array[x[0]][x[1]], reverse=True))
    
    print("Non-zero entries:")
    for i, j in zip(*non_zero_entries):
        print("Row: {}, Column: {}, Value: {}".format(i, j, array[i][j]))

In [122]:
display_non_zero(Q_value.reshape(-1,1))

Number of non-zero entries: 2175
Non-zero entries:
Row: 16070, Column: 0, Value: 9.113706588745117
Row: 4164, Column: 0, Value: 8.510761260986328
Row: 4771, Column: 0, Value: 8.499053955078125
Row: 16194, Column: 0, Value: 8.499053955078125
Row: 17654, Column: 0, Value: 8.499053955078125
Row: 1946, Column: 0, Value: 8.407470703125
Row: 2703, Column: 0, Value: 8.365306854248047
Row: 14720, Column: 0, Value: 8.357839584350586
Row: 15260, Column: 0, Value: 8.334234237670898
Row: 269, Column: 0, Value: 8.146980285644531
Row: 15185, Column: 0, Value: 8.146980285644531
Row: 4766, Column: 0, Value: 8.093132972717285
Row: 2707, Column: 0, Value: 7.941089153289795
Row: 14222, Column: 0, Value: 7.941089153289795
Row: 16190, Column: 0, Value: 7.941089153289795
Row: 18874, Column: 0, Value: 7.941089153289795
Row: 4511, Column: 0, Value: 7.882282257080078
Row: 2076, Column: 0, Value: 7.832083225250244
Row: 521, Column: 0, Value: 7.799874305725098
Row: 2063, Column: 0, Value: 7.773755073547363
Row: 

In [11]:
import numpy as np
import os

In [1]:
action_idx = 2712

In [2]:
def action2elec_amp(action_idx, n_amps):
    # convert action_idx to (elec, amp)
    (elec, amp) = (action_idx//n_amps + 1, action_idx%n_amps + 1)
    return (elec, amp)

In [3]:
def load_dictionary(path):
    # Load relevant data from .npz files
    try:
        with np.load(os.path.join(path,"dictionary.npz")) as data:
            dict = data["dictionary"]
            elecs = data["entry_elecs"]
            amps = data["entry_amps"]
            elec_map = data["elec_map"]
        with np.load(os.path.join(path,"decoders.npz")) as data:
            cell_ids = data["cell_ids"]
    except FileNotFoundError:
        print("Please make sure the dictionary.npz and decoders.npz files are present in the specified path")

    return dict, elecs, amps, elec_map, cell_ids

In [4]:
def get_dict_from_action_idx(action_idx, n_amps, path):
    # convert action_idx to (elec, amp)
    (elec, amp) = action2elec_amp(action_idx, n_amps)
    
    dict, elecs, amps, elec_map, cell_ids = load_dictionary(path)

    # get dictionary entry
    try:
        idx = np.where((elecs == elec) & (amps == amp))[0][0]
        dict_entry = dict[idx]
    except IndexError:
        print(f"Electrode {elec} with amplitude {amp} was not in the dictionary")
        print(f"Assume no cells were activated")
        dict_entry = np.zeros(len(cell_ids), dtype=np.float64)

    return dict_entry

In [12]:
get_dict_from_action_idx(action_idx, 43, path)

Electrode 64 with amplitude 4 was not in the dictionary
Assume no cells were activated


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [59]:
new_dict = []
for a in np.nonzero(Q_value)[0]:
    new_dict.append(get_dict_from_action_idx(a, sim.n_amps, path))
new_dict = np.array(new_dict)

In [60]:
new_dict.shape

(1922, 147)

In [61]:
span_score_func(new_dict)

109

In [64]:
dict = load_dictionary(path)[0]

In [65]:
dict.shape

(4702, 147)

In [76]:
def mag_cosine_sim_score_func(vectors):
    score = 0.0
    _, dim = vectors.shape
    # calculate the magnitude of each vector
    mag = np.linalg.norm(vectors, axis=1)

    # calculate the magnitude difference between each vector and unit vector
    abs_mag_diff = np.abs(mag - 1)

    # calculate the cosine similarity between each vector and unit vector
    for d in range(dim):
        unit_vec = np.zeros(dim)
        unit_vec[d] = 1
        cos_sim = np.dot(vectors, unit_vec) / mag
        score += np.max(cos_sim)-abs_mag_diff[np.argmax(cos_sim)]

    return score

In [79]:
vectors = np.array([[1,0,0],[0,0.1,0],[0,0,1]])
score = mag_cosine_sim_score_func(vectors)
print(score)

2.1


In [67]:
hoge = np.zeros(5)
hoge[0]=1
np.linalg.norm(hoge)

1.0

In [69]:
np.linalg.norm(np.array([0.1,0.05,0,0,0,1]))

1.0062305898749053

In [6]:
import numpy as np
from env.single_state_env import SingleStateEnvironment
from utils.reward_func import inverse_reward_func
from utils.score_func import span_score_func, mag_cosine_sim_score_func, cosine_sim_score_func
from utils.helper import display_non_zero, get_dict_from_action_idx, load_dictionary, action2elec_amp
from agent.epsilon_greedy import EpsilonGreedyAgent
from agent.thompson_sampling import TSAgent
from tqdm import tqdm

In [7]:
experiments = ["2022-11-04-2", "2022-11-28-1"]
path = f"./data/{experiments[0]}/dictionary"

env = SingleStateEnvironment(path, reward_func=inverse_reward_func, score_func=span_score_func, n_maxstep=537600, n_elecs=512, n_amps=42)
agent = EpsilonGreedyAgent(env, gamma=0.7, epsilon=0.8, decay_rate=1-10e-7, lr=0.1)
# agent = TSAgent(env, gamma=0.9, lr=0.1)


In [8]:
Q = agent.run(n_episodes=1000)

display_non_zero(Q, top_k=10)

Number of non-zero entries: 755
Top 10 Non-zero entries:
Row: 0, Column: 13044, Value: 3.205662727355957
Row: 0, Column: 7078, Value: 0.4284169673919678
Row: 0, Column: 1692, Value: 0.4217602610588074
Row: 0, Column: 15400, Value: 0.41635894775390625
Row: 0, Column: 20949, Value: 0.3586578965187073
Row: 0, Column: 9534, Value: 0.33181244134902954
Row: 0, Column: 4977, Value: 0.3293997645378113
Row: 0, Column: 15255, Value: 0.3288901150226593
Row: 0, Column: 952, Value: 0.32875269651412964
Row: 0, Column: 970, Value: 0.3283143937587738


In [9]:
env.get_est_dictionary()

ValueError: operands could not be broadcast together with shapes (21504,147) (21504,) 

In [24]:
np.nonzero(Q[0])[0]

array([    0,    10,    35,    89,   117,   120,   149,   184,   209,
         220,   247,   277,   304,   307,   314,   347,   385,   436,
         509,   513,   515,   522,   538,   540,   551,   555,   556,
         561,   598,   622,   701,   714,   757,   769,   770,   780,
         860,   874,   891,   916,   924,   993,   995,  1040,  1056,
        1119,  1174,  1182,  1185,  1321,  1331,  1396,  1441,  1467,
        1469,  1470,  1492,  1560,  1615,  1634,  1640,  1681,  1878,
        1927,  1971,  2040,  2062,  2101,  2103,  2117,  2188,  2221,
        2228,  2233,  2271,  2274,  2287,  2335,  2502,  2510,  2542,
        2550,  2643,  2647,  2664,  2694,  2756,  2764,  2887,  2952,
        2957,  2973,  3037,  3091,  3102,  3191,  3205,  3214,  3229,
        3239,  3252,  3258,  3296,  3301,  3326,  3354,  3397,  3412,
        3442,  3460,  3463,  3514,  3565,  3575,  3578,  3595,  3622,
        3679,  3708,  3718,  3730,  3755,  3769,  3786,  3832,  3833,
        3834,  3867,

In [22]:
Q[0,np.nonzero(Q[0])[0]]

array([0.0485    , 0.06934369, 0.05776079, 0.01521487, 0.05319454,
       0.0609802 , 0.06230542, 0.15483974, 0.05698894, 0.0595472 ,
       0.02945727, 0.01826576, 0.06050941, 0.0609802 , 0.03764339,
       0.01000061, 0.8450824 , 0.05653581, 0.07127505, 0.06532644,
       0.15619136, 0.10970059, 0.05915577, 0.07269973, 0.07101174,
       0.05483973, 0.16396384, 0.06328749, 0.01431567, 0.06427184,
       0.06942974, 0.02025961, 0.07424112, 0.06328749, 0.06396384,
       0.16138887, 0.07726617, 0.03388486, 0.16043626, 0.03651409,
       0.11469963, 0.04691172, 0.07051875, 0.03678802, 0.02945727,
       0.07490293, 0.05869412, 0.07494818, 0.07127505, 0.06204492,
       0.01252228, 0.16086921, 0.17424113, 0.06798154, 0.05894422,
       0.07490293, 0.06503512, 0.03388486, 0.06328749, 0.06817947,
       0.05483973, 0.00970059, 0.06469254, 0.05698894, 0.02025961,
       0.07025962, 0.17494819, 0.07269973, 0.05576887, 0.16798154,
       0.03792579, 0.00970059, 0.05565498, 0.06138887, 0.07127

In [ ]:
np.nonzero(Q[0])[0]